# 5. gyakorlat (Mohó algoritmusok)
Mielőtt belevágnánk az anyagba, lássuk, hogy mit kell majd tudni a jövő heti ZH-ra:

## ZH témák
- Algoritmusok elemzése (kód alapján futási idő, futási időből ordó kiszámolása, futási idők összehasonlítása, kódok értelmezésének képessége)
- Oszd meg és uralkodj, rekurzió (szöveges feladat alapján rekurzív összefüggések felírása, feladatok megoldása, hátrányok, előnyök ismerete)
- Dinamikus programozás (szöveges feladat alapján táblázat felrajzolása és kitöltése a feltárt összefüggéseknek megfelelően, hátrányok, előnyök ismerete)
- Mohó algoritmusok (algoritmusok tervezése, feladat alapján mohó stratégia végrehajtása, optimalitás eldöntése)

## Mohó algoritmusok
A mai órán a mohó problémamegoldási stratégiáról fogunk beszélni. Tipikusan optimalizálási feladatoknál használjuk ezeket (pl. az előző órai pontgyűjtős feladat: adjunk meg egy olyan útvonalat, hogy a maximális pontszámot gyűjtsük össze a célig, vagy pénzváltási feladat: hogyan tudjuk minimális számú érmével felváltani a kívánt összeget?), a stratégiánk pedig a következő lesz:

Ilyen feladatok megoldásánál minden lépésben egy döntést kell meghozni (pl. merre lépek, melyik érmét használom fel következőnek, stb.), a döntés után pedig az eredeti feladat egy részproblémáját kell tovább oldanom (pl. egy lépés megtétele után már kisebb a táblázat, amit be kell járnom, egy érme felhasználása után pedig már kisebb az összeg, amit fel kell váltanom). **A mohó stratégia lényege**, hogy minden egyes lépésben a döntés meghozatalát csakis a lokálisan elérhető lehetőségek befolyásolják, ez alapján döntök arról, hogy mit fogok csinálni (pontgyűjtő feladatnál pl. megnézem, hogy fölfelé, vagy jobbra található-e a nagyobb pontszám és arra fogok menni, amerre nagyobb, függetlenül minden más tényezőtől), magyarul **lokálisan optimális** döntést hozok. Könnyen látható, hogy ez nem mindig lesz jó megközelítés (pontosan a pontgyűjtő feladat jó példa erre, tekintsük pl az alábbi táblázatot):

                                                10  2  4  2

                                                2  5  1  3

                                                1  4  4  2
                                                
Ha mohó stratégiát használok, akkor az 1->4->5->2->4->2 útvonalat fogom választani, holott látszik, hogy a 10-es mezőt mindenképp meg kellene látogatnom, ehhez azonban az 1. lépést nem mohó módon kellene megválasztani (éppen ezért erre a problémára az előző órán nézett DP algoritmus lesz jó, nem pedig a mohó). 

Vannak azonban olyan feladatok is, amelyekre jól fog működni a mohó algoritmus is, ehhez viszont ezt bizonyítani kell tudni. Ez a következőképp néz ki:

1. Fogalmazzuk meg az optimalizálási feladatot úgy, hogy minden lépés hatására **pontosan egy** megoldandó részprobléma keletkezzen
2. Bizonyítsuk be, hogy az eredeti probléma megoldható optimálisan úgy, hogy mohó módszerrel választom ki a következő lépést
3. Bizonyítsuk be, hogy a mohó választás hatására olyan részproblémát kapunk, aminek az optimális megoldása + a mohó lépés megadja az eredeti problémának is az optimális megoldását.

Gyakorlaton a bizonyításokkal nem fogunk foglalkozni (előadásra viszont kellhet), viszont megnézünk bizonyos feladatokat, megnézzük, hogy a mohó módszer melyekre működik és melyekre nem.

### Hátizsák feladat
Egy adott hátizsákba tárgyakat akarunk pakolni. Adott $n$ tárgy, mindegyikhez tartozik egy $f_i$ (valós szám) fontossági érték és egy $s_i$ (egész szám) súlyérték (tehát az $i.$ tárgy reprezentálható az $(f_i,s_i)$ kettessel). A hátizsákba maximum összesen $S$ súlyt pakolhatunk. Szeretnénk úgy választani a tárgyakat, hogy az összfontosság maximális legyen. A feladatnak 3 népszerű variánsa létezik, mi itt kettőt fogunk vizsgálni:
- a **0-1 hátizsák feladat** megoldásakor egy tárgy vagy bekerül a hátizsákba, vagy nem.
- a **töredékes hátizsák feladat** megengedi, hogy egy tárgynak csak valamekkora részét tegyem el (pl. ha félbetöröm, akkor feleannyi lesz az értéke is)

Legyenek a feladathoz a konkrét értékek a következők:
- $S=12$
- a tárgyak: $(5,3), (4,4), (5,2), (5,6), (4,5)$

#### 0-1 probléma
##### DP megoldás
Jól megoldható DP-vel: vegyünk egy $T$ táblázatot $n$ sorral és $S+1$ oszloppal: $T[i,j]$ adja meg, hogy mennyi a maximális érték, amit berakhatunk a hátizsákba, ha az első $i$ tárgyat használhatom fel, a hátizsák kapacitása pedig $j$. Ekkor

$$T[i,j] = \max(T[i-1,j],f_i+T[i-1,j-s_i]),$$

hiszen minden lépésben eldönthetem, hogy beleveszem-e az adott tárgyat a hátizsákba, avagy sem. Értelemszerűen akkor veszem bele, ha nagyobb összértéket érek el vele, mint ha nem venném be, ezért kell $\max()$-ot számolni.

##### mohó megoldás
Számítsunk ki minden tárgyra egy $\frac{f_i}{s_i}$ egységnyi súlyra jutó értéket, majd minden lépésben válasszuk azt a tárgyat, amelyre a legnagyobb ez a hányados. Ez a megoldás nem feltétlenül optimális.

#### töredékes hátizsák
Ugyanúgy használjuk a mohó módszert, azzal a különbséggel, hogy az utoljára beférő tárgyat törjük szét úgy, hogy éppen tele legyen a hátizsák. Így már optimális a mohó algoritmus.

In [81]:
import numpy as np

# egy tárgy "egységárának" kiszámolása
def unit_price(item):
    return item[0]/item[1]


# mohó algoritmus a hátizsák problémára
def knapsack_greedy(S,items):
    # rendezzük a tárgyakat csökkenő sorrendbe egységár szerint
    items.sort(key=unit_price, reverse=True)
    
    result = 0
    i = 0
    while (S-items[i][1] >= 0):
        # pakoljuk a hátizsákba a tárgyakat, amíg van hely
        S -= items[i][1]
        result += items[i][0]
        i += 1
        
    return result

# DP algoritmus a hátizsák feladatra
def knapsack_dp(S, items):
    n = len(items)
    T = np.ndarray((n+1,S+1), dtype=int)
    
    for i in range(n+1):
        T[i,0] = 0
        
    for j in range(S+1):
        T[0,j] = 0
        
    for i in range(1,n+1):
        for j in range(S+1):
            if items[i-1][1] > j:
                T[i,j] = T[i-1,j]
            else:
                T[i,j] = max(T[i-1,j], items[i-1][0]+T[i-1,j-items[i-1][1]])
    
    return T[n,S]
    

In [82]:
print(knapsack_greedy(12, [(5,3),(4,4),(5,2),(5,6),(4,5)]))
print(knapsack_dp(12, [(5,3),(4,4),(5,2),(5,6),(4,5)]))

14
15


### Intervallum lefedés
Adott a számegyenesen valós számok $x_1, x_2, ..., x_n$ halmaza. Adjunk algoritmust, amely megad minimális egységnyi hosszú intervallumot, amelyek tartalmazzák a pontokat. Mik a lehetséges mohó stratégiák?
- Válasszunk minden lépésben olyan intervallumot, amivel a maximális számú pontot tudjuk lefedni egyszerre: **nem optimális.**
- Minden lépésben válasszuk azt az intervallumot, amelynek kezdőpontja az a legkisebb pont, ami még nincs lefedve: **optimális.**

### Módosított pénzváltás
Adott $F$ összeget szeretnénk felváltani a $ \mathbf{P} = (P_1, P_2, ..., P_n)$ érmékkel. Legkevesebb hány érmét kell felhasználnom a váltáshoz? 

**Megoldás:** mohó módon használjuk a lehető legnagyobb $P_i$ érmét, amíg $P_i \gt F$, ezt követően térjünk át $P_{i-1}$-re, ...

- Legyen $\mathbf{P} = (1, 5, 10, 25)$. Igazoljuk, hogy optimális a fenti algoritmus!
- Van olyan $\mathbf{P}$ vektor, amelyre nem optimális a mohó?